In [24]:
import pandas as pd
import os
import numpy as np


In [ ]:

base_table_name = 'aqe-nta'
df = pd.read_csv(f'data/base_tables/{base_table_name}.csv')
df.head()

In [16]:
# df.drop(columns=['PM_Avg'], inplace=True)
# df.head()

df[['PM_tertiles']].value_counts()


PM_tertiles
Low            88
High           87
Medium         87
dtype: int64

In [17]:
from oracle.classifier import ClassifierOracle

oracle = ClassifierOracle()

oracle.train(df, target_col='PM_tertiles')




0.8224053724053725

In [7]:
aug_name = 'Buildings_Selected_for_the_Alternative_Enforcement_Program_(AEP)'
df_aug = pd.read_csv(f'data/join_datasets/aqe-nta/{aug_name}.csv')
df_aug.head()


,BUILDING_ID,BOROUGH,NUMBER,STREET,TOTAL_UNITS,AEP_START_DATE,# OF B/C VIOLATIONS AT START,CURRENT_STATUS,DISCHARGE_DATE,AEP_ROUND,Postcode,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,37304,Manhattan,305,WEST 99 STREET,47,01/31/2020,401,AEP Discharged,08/17/2020,Aep Round 13,10025,40.797322,-73.972092,7,6,187,1057089.0,1.018880e+09,Upper West Side-Manhattan Valley
1,876311,Brooklyn,1575,EAST NEW YORK AVENUE,3,02/02/2015,155,AEP Discharged,09/21/2016,Aep Round 8,11212,40.672676,-73.909933,16,41,363,3391263.0,3.014610e+09,Ocean Hill
2,81496,Bronx,2118,GRAND AVENUE,3,11/12/2009,193,AEP Discharged,10/26/2016,Aep Round 3,10453,40.855688,-73.907845,5,14,251,2014166.0,2.031940e+09,University Heights (South)-Morris Heights
3,6247,Manhattan,411,AUDUBON AVENUE,25,01/31/2017,763,AEP Discharged,04/04/2018,Aep Round 10,10033,40.851526,-73.930284,12,10,269,1063730.0,1.021560e+09,Washington Heights (North)
4,9836,Manhattan,940,COLUMBUS AVENUE,12,02/01/2016,283,AEP Discharged,10/06/2016,Aep Round 9,10025,40.799394,-73.962766,7,7,193,1055966.0,1.018610e+09,Upper West Side-Manhattan Valley


In [5]:
overlap = df['NTA_NAME'].isin(df_aug['Geography']).sum()
print(f"重合度: {overlap / len(df)}")


重合度: 0.015267175572519083


In [8]:
base_table_name = 'Public_Housing'
base_df = pd.read_csv(f'data/base_tables/{base_table_name}.csv')
base_df.head()


,OBJECTID,shape,participant_code,formal_participant_name,development_code,project_name,building_name,building_number,building_type_code,building_status_type_code,...,annl_expns_amnt,annl_expns_amnt_prev_yr,pha_total_units,dev_buil_nu_entrance,ha_phn_num,ha_fax_num,ha_email_addr_text,exec_dir_phone,exec_dir_fax,exec_dir_email
0,966,POINT (-122.509481 37.867920999000034),CA052,HOUSING AUTHORITY OF COUNTY OF MARIN,CA052000001,Golden Gate Village,NaN,B-2,RW,INAPCP,...,-4,-4,494,CA052000001B-262,4154912525,4.154793e+09,KCarroll@marinhousing.org,4154912530,4.154793e+09,LJordan@marinhousing.org
1,965,POINT (-121.87320899999999 38.015911999000025),CA011,Housing Authority of the County Contra Costa,CA011500000,EL PUEBLO,El Pueblo,40,SD,INAPCP,...,-4,-4,1058,CA011500000401,9259578045,9.253720e+09,ecampbell@contracostahousing.org,9259578011,9.253720e+09,jvillarreal@contracostahousing.org
2,964,POINT (-122.541754 37.998125999000024),CA052,HOUSING AUTHORITY OF COUNTY OF MARIN,CA052000002,KRUGER PINES,NaN,GH-K,RW,INAPCP,...,-4,-4,494,CA052000002GH-K1,4154912525,4.154793e+09,KCarroll@marinhousing.org,4154912530,4.154793e+09,LJordan@marinhousing.org
3,963,POINT (-122.42763199999999 37.650152999000056),CA015,City of South San Francisco Housing Authority,CA015000001,HOUSING AUTHORITY OF THE CITY OF SSF,NaN,364,RW,INAPCP,...,-4,-4,80,CA0150000013641,6505837631,6.505836e+09,sosfha@sbcglobal.net,6505837631,6.505836e+09,sosfha@sbcglobal.net
4,962,POINT (-122.151456 38.05722399900003),CA041,CITY OF BENICIA HSG AUTH,CA041000001,CAPITOL HEIGHTS,"Building 38, 001",38,SD,INAPCP,...,-4,-4,75,CA04100000138001,7077452071,7.077458e+09,juliep@BeniciaHousingAuthority.org,7077452071,7.077458e+09,juliep@BeniciaHousingAuthority.org


## 左连接处理
- 取第一个值
- 聚合方法

In [43]:
df_aug.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4137 entries, 0 to 4136
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   BUILDING_ID                   4137 non-null   int64  
 1   BOROUGH                       4137 non-null   object 
 2   NUMBER                        4137 non-null   object 
 3   STREET                        4137 non-null   object 
 4   TOTAL_UNITS                   4137 non-null   int64  
 5   AEP_START_DATE                4137 non-null   object 
 6   # OF B/C VIOLATIONS AT START  4137 non-null   int64  
 7   CURRENT_STATUS                4137 non-null   object 
 8   DISCHARGE_DATE                3198 non-null   object 
 9   AEP_ROUND                     4137 non-null   object 
 10  Postcode                      4137 non-null   int64  
 11  Latitude                      4137 non-null   float64
 12  Longitude                     4137 non-null   float64
 13  Com

In [48]:
print(df_aug.shape)
df_agg = df_aug.groupby('NTA').agg({'NTA': 'count', 'Latitude': 'mean', 'Longitude': 'mean', 'CURRENT_STATUS': lambda x: x.mode()[0]})
print(df_agg.shape)
df_agg.head()


(4137, 19)
(160, 4)


,NTA,Latitude,Longitude,CURRENT_STATUS
NTA,,,,
Allerton,23,40.864187,-73.863963,AEP Discharged
Astoria (Central),4,40.760405,-73.924453,AEP Active
Astoria (East)-Woodside (North),4,40.760123,-73.913106,AEP Discharged
Astoria (North)-Ditmars-Steinway,5,40.777029,-73.908601,AEP Discharged
Baisley Park,1,40.674814,-73.799521,AEP Discharged


In [52]:

df_aug['CURRENT_STATUS'].value_counts().index.tolist()
df_aug['CURRENT_STATUS'].describe()


count               4137
unique                 2
top       AEP Discharged
freq                3198
Name: CURRENT_STATUS, dtype: object

In [54]:
aqe_description = "We combine data from our air quality monitoring network, NYCCAS, with other data to understand what makes one neighborhood's air quality different from another's. We found that building emissions, building density, industrial areas, and traffic density are associated with differences in air quality."
schools_description = "2008-2009 School Progress Report"
base_table_info = pd.DataFrame({
    'base_table_name': ['aqe-nta', 'schools'],
    'target_attribute': ['PM_tertiles', '2009-2010 OVERALL GRADE'],
    'description': [aqe_description, schools_description]
})

base_table_info.to_csv('data/base_table_info.csv', index=False)


In [47]:
from table_process import Table
table = Table(
    data_path=f"data/join_datasets/aqe-nta/{aug_name}.csv", 
    score=0.8, 
    dataset_name=aug_name, 
    left_column_name="NTA", 
    right_column_name="NTA", 
    size=10000
)
# 找出泄露的列
oracle.train(df_merge, target_col='PM_tertiles')
print(df.shape)
print(oracle.train(df, target_col='PM_tertiles'))
for col in [c for c in df_aug.columns if c != 'NTA']:
    print(f"column: {col}")
    df_merge_1 = df.merge(df_aug[[col,'NTA']], left_on=df_aug_join_info['left_columns_names'].values[0], right_on=df_aug_join_info['right_columns_names'].values[0], how='left')
    print(df_merge_1.shape)
    print(oracle.train(df_merge_1, target_col='PM_tertiles'))


(262, 11)
0.8224053724053725
column: BUILDING_ID
(4239, 13)
0.9908428167763206
column: BOROUGH
(4239, 13)
0.9908428167763206
column: NUMBER
(4239, 13)
0.9908428167763206
column: STREET
(4239, 13)
0.9908428167763206
column: TOTAL_UNITS
(4239, 13)
0.9908428167763206
column: AEP_START_DATE
(4239, 13)
0.9908428167763206
column: # OF B/C VIOLATIONS AT START
(4239, 13)
0.9908428167763206
column: CURRENT_STATUS
(4239, 13)
0.9908428167763206
column: DISCHARGE_DATE
(4239, 13)
0.9908428167763206
column: AEP_ROUND
(4239, 13)
0.9908428167763206
column: Postcode
(4239, 13)
0.9908428167763206
column: Latitude
(4239, 13)
0.9908428167763206
column: Longitude
(4239, 13)
0.9908428167763206
column: Community Board
(4239, 13)
0.9908428167763206
column: Council District
(4239, 13)
0.9908428167763206
column: Census Tract
(4239, 13)
0.9908428167763206
column: BIN
(4239, 13)
0.9908428167763206
column: BBL
(4239, 13)
0.9908428167763206


## 处理数据泄露

In [17]:
data_dir = os.path.join('join_datasets/', base_table_name)

for file in os.listdir(data_dir):
    df_aug = pd.read_csv(os.path.join(data_dir, file))
    for col in df_aug.columns:
        if '2009' in col:
            print(f"{file}: drop column {col}")
            df_aug.drop(col, axis=1, inplace=True)
    df_aug.to_csv(os.path.join(data_dir, file), index=False)



2006_-_2011_School_Progress_Reports_-_All_Schools_-__Multiyear_Summary.csv: drop column 2009-10 PROGRESS REPORT GRADE
2006_-_2013_Citywide_Progress_Report.csv: drop column 2009-10 PROGRESS REPORT GRADE
2007_-_2011_School_Progress_Report_Multi-year.csv: drop column 2009-10 PROGRESS REPORT GRADE
2010_-_2011_School_Progress_Report.csv: drop column 2009-10 PROGRESS REPORT GRADE
2011-2012_School_Progress_Report_-_All_Schools.csv: drop column 2009-10 PROGRESS REPORT GRADE
2011_-_2012_Citywide_Progress_Report.csv: drop column 2009-10 PROGRESS REPORT GRADE
2012_-_2013_Citywide_Progress_Report.csv: drop column 2009-10 PROGRESS REPORT GRADE
Schools_Progress_Report_2012-2013.csv: drop column 2009-10 PROGRESS REPORT GRADE


In [34]:
df_join_info = pd.read_csv(f'join_info/{base_table_name}_join_info.csv')
df_join_info.head()

aug_set = []
for f in os.listdir(data_dir):
    aug_set.append(f.split('.')[0])

display(len(aug_set))

df_join_info = df_join_info[df_join_info['dataset_name'].isin(aug_set)]


df_join_info.to_csv(f'join_info/{base_table_name}_join_info.csv', index=False)

len(df_join_info['dataset_name'])




29

29

In [13]:
from importlib_metadata import version

version('tqdm')


'4.66.5'

## ELSE

In [ ]:
ids = ['datamart.upload.976a384921d34ca1a6304a6c7bc256d7']

In [35]:
from data.get_datasets import download_dataset

id = "datamart.upload.976a384921d34ca1a6304a6c7bc256d7"

df = download_dataset(id)
df.head()

downloading datamart.upload.976a384921d34ca1a6304a6c7bc256d7...


,FIPS,State,Area,RUCCode,POVALL_2016
0,35005,NM,Chaves County,5,13974
1,13297,GA,Walton County,1,11385
2,13137,GA,Habersham County,6,6500
3,54017,WV,Doddridge County,9,1460
4,55055,WI,Jefferson County,4,7618


In [52]:
df.to_csv("data/base_tables/Poverty.csv")

In [40]:
for col_name, col in df.items():
    print(f"col name:{col_name}, na:{df[col_name].isna().sum()}, nunique:{df[col_name].nunique()}")

col name:FIPS, na:0, nunique:3136
col name:State, na:0, nunique:51
col name:Area, na:0, nunique:1871
col name:RUCCode, na:0, nunique:9
col name:POVALL_2016, na:0, nunique:2790


In [43]:
from oracle.regression import RegressionOracle

classifier = RegressionOracle()
classifier.train(df, 'POVALL_2016')

-122.81664347133757